In [ ]:
import numpy as np
import pandas as pd
from google.colab import drive

In [ ]:
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
file_path = '/content/drive/MyDrive/YTML/YTML_Project_Data/V1_merged_raw_text_data.csv'
df = pd.read_csv(file_path)

#**DataCleaning**

In [ ]:
df.head()
df.tail()

,video_id,title,published_at,channel_id,category_id,view_count,like_count,comment_count,duration,description,...,channel_start_date,subscriber_count,channel_view_count,channel_video_count,profile_picture_url,banner_image_url,publish_hour,publish_day_name,title_length,description_length
6139,smkyorC5qwc,The Third Presidential Debate: Hillary Clinton...,2016-10-20 03:10:47+00:00,UCeY0bbntWzzVIaj2z3QigXg,25,17371011,75688,28506,PT1H55M59S,» Get Breaking News Alerts: http://nbcnews.to/...,...,2006-07-19T20:46:03Z,11400000,8723780622,80351,https://yt3.ggpht.com/PJj5jtuEOi5UmkFy4IBonj5W...,https://yt3.googleusercontent.com/qwg42qXrxo1K...,3,Thursday,88,998.0
6140,1p-ggPszgfQ,"Ben Carson, Ted Cruz Respond to Iowa Voting Sc...",2016-02-07 02:37:03+00:00,UCBi2mrWuNuyYy4gbM6fU18Q,25,103928,789,175,PT5M8S,ABC News Republican Presidential Debate in New...,...,2006-08-07T23:12:21Z,18800000,16900345142,103051,https://yt3.ggpht.com/GJ8V0NX6NddGh9bf4zED4tsj...,https://yt3.googleusercontent.com/hDmrdpsLKQvR...,2,Sunday,82,139.0
6141,s67glms9mI4,The GOP Debate in Three Minutes,2015-09-17 16:45:56+00:00,UCUMZ7gohGI9HcU9VNsr2FJQ,25,161618,1178,69,PT2M39S,Sep 17 -- Eleven candidates took the stage Wed...,...,2006-03-09T23:17:35Z,4720000,1040738498,49213,https://yt3.ggpht.com/8sL0eqOeU4a1wZBVxn5NOFsu...,https://yt3.googleusercontent.com/I0w9cTvhut14...,16,Thursday,31,939.0
6142,7fmzBwnmQJM,Clinton vs. Trump: The second 2016 presidentia...,2020-09-26 15:42:36+00:00,UC6ZFN9Tx6xh-skXCuRHCDpQ,25,122273,762,0,PT1H33M50S,The 2016 presidential election cycle featured ...,...,2009-11-30T16:50:12Z,4890000,2749244571,54865,https://yt3.ggpht.com/3qIMp79Z-ynhtXTYyNcdybHw...,https://yt3.googleusercontent.com/4qqFa3wwrzSZ...,15,Saturday,54,1127.0
6143,qlaum72JNRA,The Second Presidential Debate: Hillary Clinto...,2016-10-10 14:52:42+00:00,UCvJJ_dzjViJCoLf5uKUTwoA,25,3703734,21006,9628,PT1H32M15S,» Subscribe to CNBC: http://cnb.cx/SubscribeCN...,...,2006-06-17T16:34:17Z,3980000,1888827932,15667,https://yt3.ggpht.com/rkKj6aQsYMytXxqeLCRF1Anm...,https://yt3.googleusercontent.com/o6uTwVk6yLqI...,14,Monday,85,625.0


In [ ]:
df.shape

(6144, 23)

**Check which columns have missing values:**

In [ ]:
# Calculate the number of missing values for each column
missing_values = df.isnull().sum()

# Filter the list to show only columns with at least one missing value
columns_with_missing_values = missing_values[missing_values > 0]

# Print the result
print("Columns with missing values:")
print(columns_with_missing_values)

Columns with missing values:
description            881
channel_description    313
banner_image_url       421
dtype: int64


**Fill these values**

In [ ]:

df['has_banner_image'] = np.where(df['banner_image_url'].isnull(), 0, 1)

# Corrected: Use assignment WITHOUT inplace=True
df['description'] = df['description'].fillna('')
df['channel_description'] = df['channel_description'].fillna('')
df['banner_image_url'] = df['banner_image_url'].fillna('')

# Verify that all relevant missing values are now filled
print("Missing values handled. Updated null count:")
print(df.isnull().sum())

Missing values handled. Updated null count:
video_id               0
title                  0
published_at           0
channel_id             0
category_id            0
view_count             0
like_count             0
comment_count          0
duration               0
description            0
tags                   0
channel_title          0
channel_description    0
channel_start_date     0
subscriber_count       0
channel_view_count     0
channel_video_count    0
profile_picture_url    0
banner_image_url       0
publish_hour           0
publish_day_name       0
title_length           0
description_length     0
has_banner_image       0
dtype: int64


**Get the Date and Time features**

In [ ]:
#right now the time and date columns aren't in the correct format, I will change that.
# Convert date columns using the robust ISO8601 format
df['published_at'] = pd.to_datetime(df['published_at'], format='ISO8601', errors='coerce')
df['channel_start_date'] = pd.to_datetime(df['channel_start_date'], format='ISO8601', errors='coerce')

# Extract features from the video publish date
df['publish_hour'] = df['published_at'].dt.hour
df['publish_day_of_week'] = df['published_at'].dt.dayofweek # Monday=0, Sunday=6

# Calculate the channel's age in days at the time of publishing
df['channel_age_days'] = (df['published_at'] - df['channel_start_date']).dt.days

# Verify the new columns
print("Created time-based features.")
display(df[['published_at', 'channel_start_date', 'channel_age_days', 'publish_hour']].head())

Created time-based features.


,published_at,channel_start_date,channel_age_days,publish_hour
0,2024-12-19 00:40:54+00:00,2024-07-20 23:40:34.027592+00:00,151,0
1,2024-08-16 16:08:32+00:00,2019-10-10 14:40:46.568078+00:00,1772,16
2,2025-07-21 13:02:04+00:00,2023-07-19 04:42:59.663153+00:00,733,13
3,2023-07-12 12:00:22+00:00,2023-02-26 14:58:26.165738+00:00,135,12
4,2025-02-19 11:43:05+00:00,2024-10-21 03:44:37.010170+00:00,121,11


In [ ]:
# since negative age is not possible, there seems to be a faulty row in df
# Find and display all rows where channel_age_days is negative
problematic_rows = df[df['channel_age_days'] < 0]

print(f"Found {len(problematic_rows)} rows with a negative channel age.")
display(problematic_rows[['published_at', 'channel_start_date', 'channel_age_days']])

#one row was found and then deleted

Found 1 rows with a negative channel age.


,published_at,channel_start_date,channel_age_days
4491,2008-10-14 02:07:26+00:00,2009-12-13 03:04:53+00:00,-426


In [ ]:
# Keep only rows where channel_age_days is not negative
original_row_count = len(df)
df = df[df['channel_age_days'] >= 0]


print(f"Removed {original_row_count - len(df)} problematic row.")
print(f"The new minimum channel age is now: {df['channel_age_days'].min()}")

Removed 1 problematic row.
The new minimum channel age is now: 0


In [ ]:
#create a new column 'duration_seconds'
import re

def parse_duration_to_seconds(duration_str):
    if not isinstance(duration_str, str):
        return 0
    match = re.match(r'PT(?:(\d+)H)?(?:(\d+)M)?(?:(\d+)S)?', duration_str)
    if not match:
        return 0

    hours = int(match.group(1)) if match.group(1) else 0
    minutes = int(match.group(2)) if match.group(2) else 0
    seconds = int(match.group(3)) if match.group(3) else 0

    return hours * 3600 + minutes * 60 + seconds

# Handle any missing values and apply the function
df['duration'] = df['duration'].fillna('PT0S')
df['duration_seconds'] = df['duration'].apply(parse_duration_to_seconds)

# --- Verify the result ---
print("Successfully created the 'duration_seconds' column.")
display(df[['duration', 'duration_seconds']].head())

Successfully created the 'duration_seconds' column.


,duration,duration_seconds
0,PT13S,13
1,PT5M50S,350
2,PT53S,53
3,PT42S,42
4,PT26S,26


In [ ]:
# Set pandas to display floating-point numbers in standard decimal format
pd.set_option('display.float_format', lambda x: '%.2f' % x)

Creating Target Columns

In [ ]:

# --- Classification Target (4 Tiers) ---
# Define the view count thresholds using quantiles
q25 = df['view_count'].quantile(0.25)
q75 = df['view_count'].quantile(0.75)
q95 = df['view_count'].quantile(0.95)

print(f"Thresholds:\n- Average: > {q25:,.0f}\n- Popular: > {q75:,.0f}\n- Viral: > {q95:,.0f}\n")

# Create a function to assign a video to a tier
def assign_tier(views):
    if views <= q25:
        return 0 # Underperforming
    elif views <= q75:
        return 1 # Average
    elif views <= q95:
        return 2 # Popular
    else:
        return 3 # Viral

df['engagement_class'] = df['view_count'].apply(assign_tier)


# --- Regression Target (Log of Views) ---
df['log_view_count'] = np.log1p(df['view_count'])


# --- Verify the new columns ---
print("Created target variables.\nClass Distribution:")
print(df['engagement_class'].value_counts().sort_index())
display(df[['view_count', 'log_view_count', 'engagement_class']].head())

Thresholds:
- Average: > 84,114
- Popular: > 2,421,856
- Viral: > 23,283,372

Created target variables.
Class Distribution:
engagement_class
0    1536
1    3071
2    1228
3     308
Name: count, dtype: int64


,view_count,log_view_count,engagement_class
0,5839276,15.58,2
1,349876,12.77,1
2,18443,9.82,0
3,35829,10.49,0
4,90850,11.42,1


In [ ]:
# --- Apply Log Transformation to Skewed Numerical Features ---
print("\nApplying log transformation to skewed features...")

# We use np.log1p which is equivalent to log(x+1) to handle any zero values gracefully
df['log_subscriber_count'] = np.log1p(df['subscriber_count'])
df['log_channel_view_count'] = np.log1p(df['channel_view_count'])
df['log_channel_video_count'] = np.log1p(df['channel_video_count'])

print("Log-transformed features created.")
display(df[['subscriber_count', 'log_subscriber_count']].head())


Applying log transformation to skewed features...
Log-transformed features created.


,subscriber_count,log_subscriber_count
0,46100,10.74
1,2770000,14.83
2,34400,10.45
3,10100,9.22
4,377,5.93


In [ ]:
df.head()
df.describe()

,category_id,view_count,like_count,comment_count,subscriber_count,channel_view_count,channel_video_count,publish_hour,title_length,description_length,has_banner_image,publish_day_of_week,channel_age_days,duration_seconds,engagement_class,log_view_count,log_subscriber_count,log_channel_view_count,log_channel_video_count
count,6143.00,6143.00,6143.00,6143.00,6143.00,6143.00,6143.00,6143.00,6143.00,6143.00,6143.00,6143.00,6143.00,6143.00,6143.00,6143.00,6143.00,6143.00,6143.00
mean,22.33,13606223.10,148718.36,3544.99,3162654.20,1798494912.87,8049.60,12.84,60.68,1015.25,0.93,2.93,2560.53,1240.10,1.05,12.91,12.52,18.12,6.35
std,12.46,145671903.02,900044.15,30068.03,8550996.43,7227179207.43,36417.70,5.74,21.66,1071.45,0.25,1.95,1978.43,4608.22,0.81,2.71,2.98,3.29,2.01
min,1.00,5.00,0.00,0.00,3.00,113.00,1.00,0.00,5.00,0.00,0.00,0.00,0.00,0.00,0.00,1.79,1.39,4.74,0.69
25%,20.00,84113.50,1442.50,41.00,45800.00,8713752.00,159.50,10.00,44.00,182.00,1.00,1.00,844.00,39.00,0.50,11.34,10.73,15.98,5.08
50%,23.00,448337.00,8994.00,211.00,390000.00,94187888.00,532.00,14.00,58.00,678.00,1.00,3.00,2245.00,163.00,1.00,13.01,12.87,18.36,6.28
75%,27.00,2421856.50,48505.50,1007.50,2730000.00,946563802.00,1622.00,17.00,78.00,1465.50,1.00,5.00,3950.00,768.50,1.50,14.70,14.82,20.67,7.39
max,99.00,6523764284.00,34618242.00,1208429.00,299000000.00,305313475617.00,528327.00,23.00,107.00,5000.00,1.00,6.00,20207.00,80157.00,3.00,22.60,19.52,26.44,13.18


#**Downloading the cleaned dataset.**

In [ ]:
# The DataFrame 'df' now contains all original and newly created columns.

output_path = '/content/drive/My Drive/YTML/YTML_Project_Data/V1_new_processed_data.csv'

# Save the entire DataFrame to the new file
df.to_csv(output_path, index=False)

# --- Verify the final result ---
print(f"Processed data with all columns saved to: {output_path}")
print(f"\nFinal DataFrame has {df.shape[1]} columns.")
df.info()

Processed data with all columns saved to: /content/drive/My Drive/YTML/YTML_Project_Data/V1_new_processed_data.csv

Final DataFrame has 32 columns.
<class 'pandas.core.frame.DataFrame'>
Index: 6143 entries, 0 to 6143
Data columns (total 32 columns):
 #   Column                   Non-Null Count  Dtype              
---  ------                   --------------  -----              
 0   video_id                 6143 non-null   object             
 1   title                    6143 non-null   object             
 2   published_at             6143 non-null   datetime64[ns, UTC]
 3   channel_id               6143 non-null   object             
 4   category_id              6143 non-null   int64              
 5   view_count               6143 non-null   int64              
 6   like_count               6143 non-null   int64              
 7   comment_count            6143 non-null   int64              
 8   duration                 6143 non-null   object             
 9   description        